# Exercise 1

a) No, we do not observe equal values for maximal activities along a linear pathway, while fluxes are forced to be equal in steady state due to mass balance constraints, maximal activities are instrinsic enzyme properties and this differs across different reactions, this is why reaction in the same linear pathway display varying activities rather than identical ones.
b) EX_gln__L_e,EX_pyr_e
	Exchange reactions dont represent enzymatic steps inside the cell but rather material flow across the system boundary. Maximal activity is derived from enzyme expression, exchange reactions are not catalysed by specific intracellular enzymes, therefore, there is no associated gene to the reaction and thus, no maximal activity.


# Exercise 2

In [44]:
import cobra
from cobra.io import load_json_model
import pandas as pd

In [36]:

# Load model
model = load_json_model("e_coli_core.json")

max_activity = pd.read_csv("e_coli_core_expression.csv")
max_activity.columns = [col.strip() for col in max_activity.columns]

max_activity_dict = dict(zip(max_activity["# Reaction ID"], max_act["reaction activity [mmol/gDW/h]"]))

for reaction_key in max_act_dict:
    # try:
        reaction = model.reactions.get_by_id(reaction_key)
        value = max_act_dict[reaction_key]
    # Change reversible reactions bounds
        if reaction.reversibility:
            reaction.upper_bound = value
            reaction.lower_bound = -value
    # Change irreversible reaction bounds
        else:
            reaction.upper_bound = value
            reaction.lower_bound = 0
    # except KeyError:
    #     pass
             
        

        
    
        

In [43]:
# glucose exhange reaction
glucose_ex = model.reactions.get_by_id("EX_glu__L_e")
print(glucose_ex.upper_bound)
print(glucose_ex.lower_bound)

# We just need to change lower bound as upper bound is already 1000

glucose_ex.lower_bound = -1000


# ATPM 
atpm = model.reactions.get_by_id("ATPM")

for reac in model.reactions:
    print(f"{reac.id} uppper bound: {reac.upper_bound}, lower bound: {reac.lower_bound}")
    
                       

1000.0
-1000
PFK uppper bound: 12.12, lower bound: 0
PFL uppper bound: 1.0, lower bound: 0
PGI uppper bound: 13.12, lower bound: -13.12
PGK uppper bound: 23.13, lower bound: -23.13
PGL uppper bound: 8.12, lower bound: 0
ACALD uppper bound: 1.16, lower bound: -1.16
AKGt2r uppper bound: 3.1, lower bound: -3.1
PGM uppper bound: 20.01, lower bound: -20.01
PIt2r uppper bound: 6.03, lower bound: -6.03
ALCD2x uppper bound: 9.01, lower bound: -9.01
ACALDt uppper bound: 2.29, lower bound: -2.29
ACKr uppper bound: 1.19, lower bound: -1.19
PPC uppper bound: 2.56, lower bound: 0
ACONTa uppper bound: 25.35, lower bound: -25.35
ACONTb uppper bound: 25.35, lower bound: -25.35
ATPM uppper bound: 1000.0, lower bound: 8.39
PPCK uppper bound: 25.23, lower bound: 0
ACt2r uppper bound: 3.23, lower bound: -3.23
PPS uppper bound: 2.5, lower bound: 0
ADK1 uppper bound: 30.57, lower bound: -30.57
AKGDH uppper bound: 24.35, lower bound: 0
ATPS4r uppper bound: 60.5, lower bound: -60.5
PTAr uppper bound: 4.47, lo

# Exercise 3